***GENERATED CODE FOR washingtonian PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateAware', 'transformation_label': 'String Indexer'}], 'feature': 'DateAware', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '479', 'mean': '', 'stddev': '', 'min': '1/1/2020 0:00', 'max': '9/9/2020 0:00', 'missing': '21', 'distinct': '319'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DateAware'}, {'feature_label': 'DateAware', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DateAware')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateSubmitted', 'transformation_label': 'String Indexer'}], 'feature': 'DateSubmitted', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1/10/2018 0:00', 'max': '9/9/2020 0:00', 'missing': '0', 'distinct': '340'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DateSubmitted'}, {'feature_label': 'DateSubmitted', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DateSubmitted')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DataBreachCause', 'transformation_label': 'String Indexer'}], 'feature': 'DataBreachCause', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Cyberattack', 'max': 'Unauthorized Access', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DataBreachCause'}, {'feature_label': 'DataBreachCause', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DataBreachCause')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateStart', 'transformation_label': 'String Indexer'}], 'feature': 'DateStart', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '417', 'mean': '', 'stddev': '', 'min': '1/1/2020 0:00', 'max': '9/7/2019 0:00', 'missing': '83', 'distinct': '266'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DateStart'}, {'feature_label': 'DateStart', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DateStart')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateEnd', 'transformation_label': 'String Indexer'}], 'feature': 'DateEnd', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '383', 'mean': '', 'stddev': '', 'min': '1/1/2017 0:00', 'max': '9/7/2020 0:00', 'missing': '117', 'distinct': '223'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DateEnd'}, {'feature_label': 'DateEnd', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DateEnd')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Name', 'transformation_label': 'String Indexer'}], 'feature': 'Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '20/20 Eye Care Network, Inc.', 'max': 'thyssenkrupp Materials', 'missing': '0', 'distinct': '495'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Name'}, {'feature_label': 'Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CyberattackType', 'transformation_label': 'String Indexer'}], 'feature': 'CyberattackType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '345', 'mean': '', 'stddev': '', 'min': 'Malware', 'max': 'Ransomware', 'missing': '155', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CyberattackType'}, {'feature_label': 'CyberattackType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CyberattackType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'IndustryType', 'transformation_label': 'String Indexer'}], 'feature': 'IndustryType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Business', 'max': 'Non-Profit/Charity', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'IndustryType'}, {'feature_label': 'IndustryType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('IndustryType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'BusinessType', 'transformation_label': 'String Indexer'}], 'feature': 'BusinessType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '235', 'mean': '', 'stddev': '', 'min': 'Accessories', 'max': 'Web Services', 'missing': '265', 'distinct': '23'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'BusinessType'}, {'feature_label': 'BusinessType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('BusinessType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'WashingtoniansAffectedRange', 'transformation_label': 'String Indexer'}], 'feature': 'WashingtoniansAffectedRange', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1,000,000+', 'max': 'Unknown', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'WashingtoniansAffectedRan...'}, {'feature_label': 'WashingtoniansAffectedRange', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'WashingtoniansAffectedRange')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'BreachLifecycleRange', 'transformation_label': 'String Indexer'}], 'feature': 'BreachLifecycleRange', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0', 'max': 'Unknown', 'missing': '0', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'BreachLifecycleRange'}, {'feature_label': 'BreachLifecycleRange', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('BreachLifecycleRange')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DaysOfExposureRange', 'transformation_label': 'String Indexer'}], 'feature': 'DaysOfExposureRange', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0', 'max': 'Unknown', 'missing': '0', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DaysOfExposureRange'}, {'feature_label': 'DaysOfExposureRange', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DaysOfExposureRange')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run washingtonianHooks.ipynb
try:
	#sourcePreExecutionHook()

	databreachnotificationsaffectingwashingtonresidents = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Data_BreachNotificationsAffectingWashington_Residents_20240219.csv', 'filename': 'Data_BreachNotificationsAffectingWashington_Residents_20240219.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(databreachnotificationsaffectingwashingtonresidents)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run washingtonianHooks.ipynb
try:
	#transformationPreExecutionHook()

	washingtonianautofe = TransformationMain.run(databreachnotificationsaffectingwashingtonresidents,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "DateAware", "transformation_label": "String Indexer"}], "feature": "DateAware", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "479", "mean": "", "stddev": "", "min": "1/1/2020 0:00", "max": "9/9/2020 0:00", "missing": "21", "distinct": "319"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DateAware"}, {"transformationsData": [{"feature_label": "DateSubmitted", "transformation_label": "String Indexer"}], "feature": "DateSubmitted", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1/10/2018 0:00", "max": "9/9/2020 0:00", "missing": "0", "distinct": "340"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DateSubmitted"}, {"transformationsData": [{"feature_label": "DataBreachCause", "transformation_label": "String Indexer"}], "feature": "DataBreachCause", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Cyberattack", "max": "Unauthorized Access", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DataBreachCause"}, {"transformationsData": [{"feature_label": "DateStart", "transformation_label": "String Indexer"}], "feature": "DateStart", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "417", "mean": "", "stddev": "", "min": "1/1/2020 0:00", "max": "9/7/2019 0:00", "missing": "83", "distinct": "266"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DateStart"}, {"transformationsData": [{"feature_label": "DateEnd", "transformation_label": "String Indexer"}], "feature": "DateEnd", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "383", "mean": "", "stddev": "", "min": "1/1/2017 0:00", "max": "9/7/2020 0:00", "missing": "117", "distinct": "223"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DateEnd"}, {"transformationsData": [{"feature_label": "Name", "transformation_label": "String Indexer"}], "feature": "Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "20/20 Eye Care Network, Inc.", "max": "thyssenkrupp Materials", "missing": "0", "distinct": "495"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Name"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "10123.2", "stddev": "346.39", "min": "9561", "max": "10554", "missing": "0"}, "updatedLabel": "Id"}, {"transformationsData": [{"feature_label": "CyberattackType", "transformation_label": "String Indexer"}], "feature": "CyberattackType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "345", "mean": "", "stddev": "", "min": "Malware", "max": "Ransomware", "missing": "155", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CyberattackType"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "WashingtoniansAffected", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "476", "mean": "28952.03", "stddev": "187866.33", "min": "15", "max": "3243664", "missing": "24"}, "updatedLabel": "WashingtoniansAffected"}, {"transformationsData": [{"feature_label": "IndustryType", "transformation_label": "String Indexer"}], "feature": "IndustryType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Business", "max": "Non-Profit/Charity", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "IndustryType"}, {"transformationsData": [{"feature_label": "BusinessType", "transformation_label": "String Indexer"}], "feature": "BusinessType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "235", "mean": "", "stddev": "", "min": "Accessories", "max": "Web Services", "missing": "265", "distinct": "23"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "BusinessType"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2019.53", "stddev": "1.86", "min": "2016", "max": "2021", "missing": "0"}, "updatedLabel": "Year"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "YearText", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2019.53", "stddev": "1.86", "min": "2016", "max": "2021", "missing": "0"}, "updatedLabel": "YearText"}, {"transformationsData": [{"feature_label": "WashingtoniansAffectedRange", "transformation_label": "String Indexer"}], "feature": "WashingtoniansAffectedRange", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1,000,000+", "max": "Unknown", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "WashingtoniansAffectedRan..."}, {"transformationsData": [{"feature_label": "BreachLifecycleRange", "transformation_label": "String Indexer"}], "feature": "BreachLifecycleRange", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "Unknown", "missing": "0", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "BreachLifecycleRange"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DaysToContainBreach", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "368", "mean": "3.7", "stddev": "15.12", "min": "0", "max": "170", "missing": "132"}, "updatedLabel": "DaysToContainBreach"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DaysToIdentifyBreach", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "399", "mean": "166.16", "stddev": "220.34", "min": "0", "max": "2039", "missing": "101"}, "updatedLabel": "DaysToIdentifyBreach"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DaysBreachLifecycle", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "363", "mean": "178.06", "stddev": "224.35", "min": "0", "max": "2039", "missing": "137"}, "updatedLabel": "DaysBreachLifecycle"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DiscoveredInProgress", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.29", "stddev": "0.46", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "DiscoveredInProgress"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DaysOfExposure", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "378", "mean": "138.6", "stddev": "228.59", "min": "0", "max": "2039", "missing": "122"}, "updatedLabel": "DaysOfExposure"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DaysElapsedBetweenEndAndDiscovery", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "222", "mean": "74.57", "stddev": "68.08", "min": "1", "max": "564", "missing": "278"}, "updatedLabel": "DaysElapsedBetweenEndAndD..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "EndedOnDayDiscovered", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.16", "stddev": "0.37", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "EndedOnDayDiscovered"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DaysElapsedBeforeNotification", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "479", "mean": "67.11", "stddev": "70.61", "min": "0", "max": "638", "missing": "21"}, "updatedLabel": "DaysElapsedBeforeNotifica..."}, {"transformationsData": [{"feature_label": "DaysOfExposureRange", "transformation_label": "String Indexer"}], "feature": "DaysOfExposureRange", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "Unknown", "missing": "0", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DaysOfExposureRange"}]}))

	#transformationPostExecutionHook(washingtonianautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run washingtonianHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(washingtonianautofe, ["Id", "Year", "YearText", "DaysToContainBreach", "DaysToIdentifyBreach", "DaysBreachLifecycle", "DiscoveredInProgress", "DaysOfExposure", "DaysElapsedBetweenEndAndDiscovery", "EndedOnDayDiscovered", "DaysElapsedBeforeNotification", "DateAware_stringindexer", "DateSubmitted_stringindexer", "DataBreachCause_stringindexer", "DateStart_stringindexer", "DateEnd_stringindexer", "Name_stringindexer", "CyberattackType_stringindexer", "IndustryType_stringindexer", "BusinessType_stringindexer", "WashingtoniansAffectedRange_stringindexer", "BreachLifecycleRange_stringindexer", "DaysOfExposureRange_stringindexer"], "WashingtoniansAffected")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

